In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { buildParser } from '@lezer/generator';
import { TreeCursor } from '@lezer/common';
import { display } from "tslab";
import { 
    AST, NumNode, VarNode, BinaryExpr, 
    cstToAST, ParserConfig, ast2dot 
} from "./AST2Dot";
import { instance } from "@viz-js/viz";
const viz = await instance();

# Resolving Conflicts Using *Precedence Declarations*

This file shows how *shift/reduce* and *reduce/reduce* conflicts can be resolved using *operator precedence declarations*.
The following grammar is *ambiguous* because it does not specify the precedence of the arithmetical operators:
```
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | expr '^' expr
         | '(' expr ')'
         | NUMBER      
         ;
```
Instead of restructuring the grammar (stratification), we use Lezer's `@precedence` block to explicitly define:
1.  **Priority:** Multiplication binds tighter than addition.
2.  **Associativity:** `^` is right-associative (`@right`), others are left-associative (`@left`).

In [ ]:
const grammarPrecedence = `
    @precedence {
        power @right,  // Right-associative: 2^3^4 -> 2^(3^4)
        times @left,   // Left-associative:  2*3/4 -> (2*3)/4
        plus  @left    // Left-associative:  1+2-3 -> (1+2)-3
    }

    @top Program { Expr }

    @tokens {
        Number { "0" | $[1-9] $[0-9]* }
        OpPlus { "+" } OpMinus { "-" }
        OpMult { "*" } OpDiv   { "/" }
        OpPow  { "^" }
        space { $[ \t\n\r]+ }
        "(" ")"
    }

    @skip { space }

    Expr {    
        Expr !plus  OpPlus  Expr |
        Expr !plus  OpMinus Expr |  
        Expr !times OpMult  Expr |
        Expr !times OpDiv   Expr |
        Expr !power OpPow   Expr |
        "(" Expr ")" |
        Number
    }
`;

const parser = buildParser(grammarPrecedence);
console.log("SUCCESS: Parser generated using Precedence Declarations.");

In [ ]:
const config: ParserConfig = {
    ignore: new Set(["(", ")"]),
    rules: {
        "Number": (_, text) => new NumNode(parseInt(text)),
        "OpPlus": () => new VarNode("+"), "OpMinus": () => new VarNode("-"),
        "OpMult": () => new VarNode("*"), "OpDiv":   () => new VarNode("/"),
        "OpPow":  () => new VarNode("^"),
        "Program": (children) => children[0],
        "Expr": (children) => {
            if (children.length === 1) return children[0];
            if (children.length === 3) {
                const opNode = children[1];
                if (opNode instanceof VarNode) {
                    return new BinaryExpr(children[0], opNode.name, children[2]);
                }
            }         
            throw new Error(`Unexpected Expr structure with length ${children.length}`);
        }
    }
};

In [ ]:
async function test(input: string) {
    try {
        const tree = parser.parse(input);
        const ast = cstToAST(tree.cursor(), input, config);        
        console.log(`Input: ${input}`);
        console.log(`Text:  ${ast.toString()}`); 
        const dot = ast2dot(ast);
        display.html(viz.renderString(dot, { format: "svg" }));
        
    } catch (e) {
        console.error(e);
    }
}

In [ ]:
await test('2^3^4*5+6-7/8^9')

In [ ]:
await test('1+2*3^4')

In [ ]:
await test('1 * 2 + (3^4)^5')